In [ ]:
pip install pyTelegramBotAPI

In [ ]:
#!/usr/bin/python

import telebot
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import pickle
from keras.models import load_model
import os

## Бот

In [ ]:
#!/usr/bin/python

import telebot
import os
from telebot import types 

API_TOKEN = # PUT YOUR API TOKEN #

bot = telebot.TeleBot(API_TOKEN)


# загрузка изображения, пэддинг, подготовка к модели, ОДИН ЦВЕТОВОЙ КАНАЛ
def load_image_grayscale(filename):
  img = tf.keras.utils.load_img(filename, grayscale=True, target_size=(28, 28))
  img = np.asarray(img).reshape(1, 28, 28, 1)
  img = img.astype('float32')
  img = tf.pad(img, paddings=tf.constant([[0,0], [2, 2,], [2, 2], [0,0]]), 
             mode ='CONSTANT') / 255.0
  return img

# предикт
def predict_digit(image_name, model_name):
  loaded_model = load_model(model_name)
  return np.argmax(loaded_model.predict(load_image_grayscale(image_name)))
# КОД БОТА #
 

@bot.message_handler(commands=['start'])
def start(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Поздороваться")
    btn2 = types.KeyboardButton("Сделать что-то")
    markup.add(btn1, btn2)
    bot.send_message(message.chat.id,
                     text="Привет, {0.first_name}!".format(message.from_user), reply_markup=markup)



@bot.message_handler(content_types=['photo'])
def handle_docs_photo(message):
    model_name = 'custom_model.h5'
    chat_id = message.chat.id
    file_info = bot.get_file(message.photo[-1].file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    src = '/content/' + message.photo[-1].file_id + '.jpg';
    with open(src, 'wb') as new_file:
      new_file.write(downloaded_file)
    result = "Это цифра " + str(predict_digit(src, model_name))
    bot.reply_to(message, result)
    if os.path.isfile(src):
      os.remove(src)


@bot.message_handler(content_types=['text', 'photo'])
def func(message):
    if (message.text == "Поздороваться"):
        bot.send_message(message.chat.id, text="Eще раз привет 😼")
    elif (message.text == "Сделать что-то"):
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        btn1 = types.KeyboardButton("Распознавание цифр")
        back = types.KeyboardButton("Вернуться в главное меню")
        markup.add(btn1, back)
        bot.send_message(message.chat.id, text="Вот что я умею:", reply_markup=markup)

    elif (message.text == "Распознавание цифр"):
        bot.send_message(message.chat.id, '''
        Пришлите цифру на однородном фоне, расположенную примерно по центру картинки
        ''')

    elif (message.text == "Вернуться в главное меню"):
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        button1 = types.KeyboardButton("Поздороваться")
        button2 = types.KeyboardButton("Сделать что-то")
        markup.add(button1, button2)
        bot.send_message(message.chat.id, text="Вы вернулись в главное меню", reply_markup=markup)
    else:
        bot.send_message(message.chat.id, text=message.text)


# ответ на ;тект; ;текстом;
@bot.message_handler(func=lambda message: True)
def echo_message(message):
    bot.reply_to(message, message.text)


bot.infinity_polling()